In [1]:
pip install gensim # pour le doc2vec

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


IMPLEMENTATION OF IMPORTS

In [6]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bs4 import BeautifulSoup
from collections import Counter
import csv
from dateutil import parser
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import pandas as pd
import random
import re
import requests
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
import string
import time
from urllib.parse import urljoin
from urllib.parse import urlparse
from wordcloud import WordCloud

In [7]:
nltk.download('punkt_tab') 
nltk.download('stopwords') 
nltk.download('wordnet')   
nltk.download('omw-1.4')   

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

IMPLEMENTATION OF CONSTANTS

In [8]:
# --- CONFIGURATION OF COLORS ---

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
RESET = "\033[0m"

In [ ]:
# --- CONFIGURATION OF STOPWORDS ---

custom_stopwords = {
    # High-frequency but uniteresting
    "one", "two", "first", "second", "new", "more", "most", "many", "much",
    "even", "still", "just", "really", "very", "well", "good", "great", "better",
    "every", "each", "some", "any", "another", "few", "several", "also", "like", "anti", "something",

    # Adverbes
    "often", "sometimes", "usually", "typically", "generally", "likely", "probably",
    "maybe", "perhaps", "always", "never", "already", "again", "soon", "now", "then",
    "various", "numerous", "multiple", "today", "yesterday", "tomorrow",
    "currently", "recently", "eventually", "rarely", "occasionally",

    # Generic verbs (not in standard stopwords)
    "get", "got", "make", "made", "take", "took", "use", "used", "using", "help", "helps",
    "see", "seen", "know", "known", "show", "shows", "say", "says", "said", "find", "found",

    # Generic nouns with low semantic value 
    "thing", "things", "stuff", "kind", "kinds", "type", "types", "way", "ways",
    "part", "parts", "area", "areas", "aspect", "aspects", "place", "places",
    "lot", "bit", "time", "day", "week", "month", "year",
    "someone", "anyone", "everyone", "nobody",

    # Mots de liaison ou de structure
    "however", "therefore", "thus", "though", "although", "because", "while", "since",
    "before", "after", "during", "when", "where", "whose", "which", "who", "whom",

    # Names
    "sarah", "regan", "amanda", "shayne", "kelly", "gansalves", "abby", "moore", "jason", "wachob", "julie", 
    "nguyen", "braelyn", "wood", "alisa", "cowell", "alexandra", "engler", "ava", "durgin", "carleigh", "ferrante", 
    "hannah", "margaret", "allen", "molly", "knudsen", "emma", "engler", "devon", "barrow", "eliza", "sullivan", 
    "loewe", "jamie", "scheinder", "frye", "lindsay", "boyers", "kelsea", "samson", "editorial", "jamie", "scheinder", "ryan", 
    "brady", "megan", "falk", "julia", "guerra", "stephanie", "eckelkamp", "india", "edwards"
}

In [ ]:
# --- CONFIGURATION OF DATE EXTRACTION ---

date_patterns = [
        # Detect dates like : "September 17, 2024" and "Sep 17, 2024"
        r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|'
        r'Jul(?:y)?|Aug(?:ust)?|Sep(?:t(?:ember)?)?|Oct(?:ober)?|Nov(?:ember)?|'
        r'Dec(?:ember)?)\s\d{1,2},\s\d{4}\b',

        # Detect dates like : "September 17 2024" 
        r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|'
        r'Jul(?:y)?|Aug(?:ust)?|Sep(?:t(?:ember)?)?|Oct(?:ober)?|Nov(?:ember)?|'
        r'Dec(?:ember)?)\s\d{1,2}\s\d{4}\b',

        # Detect dates like : "17 September 2024"
        r'\b\d{1,2}\s(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|'
        r'Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:t(?:ember)?)?|Oct(?:ober)?|'
        r'Nov(?:ember)?|Dec(?:ember)?)\s\d{4}\b',

        # Detect dates like : "2024-09-17" (ISO)
        r'\b\d{4}-\d{2}-\d{2}\b',

        # Detect dates like : "09/17/2024" and "9/17/24" 
        r'\b\d{1,2}/\d{1,2}/\d{2,4}\b'
    ]

## Faut trouver un titre pour clean, norm, tok 

IMPLEMENTATION OF DEFINITIONS

In [ ]:
def get_filenames(source_name): 
    # Create all files from a source using the name specified in the source cell [21]
    
    directory_name = "data"
    
    if not os.path.exists(directory_name): # Checks if the 'data' folder exists
        os.makedirs(directory_name) # Setup the output directory
        print(f"{GREEN}Created directory : {directory_name}{RESET}")
    
    # We assume that source_name contains no spaces. If changed, add : source_name = source_name.replace(" ", "_")

    return {"raw":          os.path.join(directory_name, f"{source_name}_raw_html.csv"),
            'content_date': os.path.join(directory_name, f"{source_name}_content_date.csv"),
            "corpus":       os.path.join(directory_name, f"{source_name}_corpus.csv"),
            "cleaned":      os.path.join(directory_name, f"{source_name}_cleaned.csv"),
            "norm":         os.path.join(directory_name, f"{source_name}_corpus_norm.csv"),
            "tokenized":    os.path.join(directory_name, f"{source_name}_corpus_tokenized.csv"),
            "nodes" :       os.path.join(directory_name, f"{source_name}_nodes.csv"),
            "edges" :       os.path.join(directory_name, f"{source_name}_edges.csv")}

Date extraction

In [11]:
def extract_date_from_html(html):

    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ", strip=True)

    for pattern in date_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE) # Find all date strings in the text that match one of the date pattern

        for match in matches:  
            try: # Try to convert the matched string into a date object to have all the dates with the same format
                date = parser.parse(match, fuzzy = False) # Parse convert the match object into a datetime with a strict parsing (fuzzy = False -> takes only set apart dates, not dates in the text) 

                if 1990 <= date.year <= 2030: # Keep only dates in a reasonable year range
                    return date

            except:
                continue # If parsing fails, ignore this match and move to the next match

    return None


In [ ]:
def content_date_csv_file(input_csv, output_csv, domain):

    df = pd.read_csv(input_csv) # Take the csv file with the raw html as the input

    if 'html' not in df.columns: # Verify that the html column exists
        raise ValueError(f"The html column is missing in: {input_csv}")
    
    df = df[df['url'].str.startswith(domain)] # We want the content of the internal links (at least for mind body green)
    
    df['date'] = df['html'].apply(extract_date_from_html) # Extract date from the html column

    df = df[['url', 'date', 'html']] # Keep columns in the desired order with the 3 columns

    df.to_csv(output_csv, index=False, encoding='utf-8') # Create a new csv file

    return df

Corpus extraction

In [ ]:
def get_corpus(html):
    # Converts an HTML text string for Python
    
    soup = BeautifulSoup(html, "html.parser") # Stores the result 
    paragraphs = [p.get_text(separator=" ", strip=True) for p in soup.find_all("p") if len(p.get_text(strip=True)) > 10]
    text = " ".join(paragraphs)

    return text

In [ ]:
def extract_corpus_csv_file(input_csv, output_csv):

    df = pd.read_csv(input_csv) # Take the csv file with the raw html as the input

    if 'html' not in df.columns: # Verify that the html colon exists
        raise ValueError(f"The html column is missing in: {input_csv}")

    df['corpus_text'] = df['html'].apply(get_corpus) # Cleans the html column

    if 'date' in df.columns :
        df = df[['url', 'date', 'corpus_text']] # Keep the url and text colon (not the raw html)
    else :
        df = df[['url', 'corpus_text']]

    df.to_csv(output_csv, index=False, encoding='utf-8') # Creats a new csv file as the output of the function

    return df

Cleaning

In [15]:
def clean_html(html):

    soup = BeautifulSoup(html, 'html.parser')

    for tag in soup(['script', 'style', 'noscript']): # Supress any unessecary tags
        tag.decompose()

    text = soup.get_text(separator=' ', strip=True) # Collect all visible text
    text = re.sub(r'\s+', ' ', text) # Supress any unecessary spaces

    return text

In [16]:
def clean_csv_file(input_csv, output_csv):

    df = pd.read_csv(input_csv) # Take the csv file with the raw html as the input

    if 'corpus_text' not in df.columns: # Verify that the html colon exists
        raise ValueError(f"The corpus column is missing in: {input_csv}")

    df['cleaned_text'] = df['corpus_text'].apply(clean_html) # Cleans the html column

    if 'date' in df.columns :
        df = df[['url', 'date', 'cleaned_text']] # Keep the url and text colon (not the raw html)
    else :
        df = df[['url', 'cleaned_text']]

    df.to_csv(output_csv, index=False, encoding='utf-8') # Creats a new csv file as the output of the function

    return df

Normalization

In [17]:
def normalize_html(text):

    text = text.lower() # Convert all characters to lowercase
    text = re.sub(r'\[\d+\]', ' ', text) # Remove reference markers like [1], [23], etc.
    text = re.sub(r'[^a-z\s]', ' ', text) # Keep only English letters and spaces (remove ponctuation, numbers, and everything that is not a letter)
    text = re.sub(r'\s+', ' ', text) # Replace multiple spaces with a single space

    return text.strip() 

In [ ]:
def normalize_csv_file(input_csv, output_csv):

    df = pd.read_csv(input_csv) # Take the csv file with the cleaned text as the input

    if 'cleaned_text' not in df.columns: # Verify that the cleaned text colon exists
        raise ValueError(f"The cleaned text is missing in: {input_csv}")

    df['normalized_text'] = df['cleaned_text'].apply(normalize_html) # Normalize the cleaned text

    if 'date' in df.columns :
        df = df[['url', 'date', 'normalized_text']] # Keep the url and normalized text 
    else : 
        df = df[['url', 'normalized_text']]

    df.to_csv(output_csv, index=False, encoding='utf-8') # Creates a new csv file as the output of the function

    return df

Tokenization

In [ ]:
def tokenize_html(text):

    stop_words = list(set(stopwords.words('english'))) + ["'s"] # Load English stopwords with an "s" at their ending
    #stem = nltk.stem.SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()

    tokens = nltk.word_tokenize(text) # Transform the text into a list of words
    tokens = [token for token in tokens if token not in stop_words] # Remove stopwords
    #tokens = [stem.stem(token) for token in tokens] # Apply stemming (racinisation)
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Apply lemmatization
    #tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    tokens = [t for t in tokens if len(t) > 1] # Remove very short or meaningless tokens 
    tokens = [t for t in tokens if len(t) < 20] # Remove very long or meaningless tokens 
    tokens = [t for t in tokens if t not in custom_stopwords] 

    return tokens

In [ ]:
def tokenize_csv_file(input_csv, output_csv):

    df = pd.read_csv(input_csv) # Take the csv file with the normalized text as the input

    if 'normalized_text' not in df.columns: # Verify that the normalized text colon exists
        raise ValueError(f"The normalized text is missing in: {input_csv}")

    df['tokenized_text'] = df['normalized_text'].apply(tokenize_html) # Cleans the html colon

    if 'date' in df.columns :
        df = df[['url', 'date', 'tokenized_text']] # Keep the url and text colon (not the raw html)
    else :
        df = df[['url', 'tokenized_text']]

    df.to_csv(output_csv, index=False, encoding='utf-8') # Creates a new csv file as the output of the function

    return df

CONFIGURATION

"""A CHANGER CAR Y A UN DOMAINE POUR FEEDSPOT ICI JE SUPPOSE"""

In [ ]:
sources = [
    {"name": "MBG",
     "domain": "https://www.mindbodygreen.com/"}, # We write the specific function without calling it ()
    {"name": "wiki",
     "domain": None}, # Wikipedia does not require domain filtering
    {"name": "blogs",
     "domain": None} 
    ]

EXECUTION

In [ ]:
"""========================== SOURCES PIPELINE =========================="""

"""=== SETUP ==="""
all_tokenized_dicts = {} 

for source in sources:
    source_name = source["name"]
    domain = source["domain"]

    files = get_filenames(source_name)

    print(f"=== SOURCE PROCESSING : {source_name} ===")
    
    print("\n=== DATES EXTRACTION ===")
    
    if domain:
        print(f"{BLUE}[{source_name}] Extracting dates and filtering by domain...{RESET}")
        content_date_csv_file(files["raw"], files["content_date"], domain=domain)
        print(f"{GREEN}Content with dates saved to {files['content_date']}{RESET}")

        input_for_corpus = files["content_date"]
    else:
        input_for_corpus = files["raw"]

    print("\n=== CORPUS EXTRACTION ===")

    print(f"{BLUE}[{source_name}] Extracting the corpus...{RESET}")
    extract_corpus_csv_file(input_for_corpus, files["corpus"])
    print(f"{GREEN}Corpus data saved to {files['corpus']}{RESET}")

    print("\n=== CLEANING ===")

    print(f"{BLUE}[{source_name}] Cleaning the data...{RESET}")
    clean_csv_file(files["corpus"], files["cleaned"])
    print(f"{GREEN}Cleaned text saved to {files['cleaned']}{RESET}")

    print("\n=== NORMALIZATION ===")

    print(f"{BLUE}[{source_name}] Normalizing the data...{RESET}")
    normalize_csv_file(files["cleaned"], files["norm"])
    print(f"{GREEN}Normalized text saved to {files['norm']}{RESET}")

    print("\n=== TOKENIZATION ===")

    print(f"{BLUE}[{source_name}] Tokenizing text...{RESET}")
    df_tokenized = tokenize_csv_file(files["norm"], files["tokenized"])
    print(f"{GREEN}Tokenized data saved in {files['tokenized']}. Rows: {len(df_tokenized)}{RESET}\n")

    current_dict = dict(zip(df_tokenized['url'], df_tokenized['tokenized_text'])) # We transform the csv file with the tokenized text into a dictionnary to make it feet the function above seen in class
    all_tokenized_dicts[source_name] = current_dict

## Term document matrix

IMPLEMENTATION OF DEFINITIONS

In [ ]:
def build_term_document_matrix(documents, min_doc_freq=2, max_doc_freq=None):

    # Step 1: Build a vocabulary (unique terms across all documents)
    vocabulary = set(token for tokens in documents.values() for token in tokens)  # The vocabulary contains all tokens appearing in the corpus. Each token is listed only once, even if it appears multiple times.

    # Step 2: Count term frequencies for each document
    term_frequencies = {doc: Counter(tokens) for doc, tokens in documents.items()}  # The dictionnary has the name of a document as a key and a counter of token frequency as value

    # Step 3: Construct the term-document matrix (a list of dictionnaries that is transform into pandas dataframe where each column corresponds to a token, each row to a document. Each dictionnary has a token as key and a list of the frequency of its number of appears in all documents as value)
    td_matrix = pd.DataFrame(
        {term: [term_frequencies[doc].get(term, 0) for doc in documents] for term in vocabulary},
        index = documents.keys())  

    # Step 4: Filter terms that appear in fewer / more than x documents
    document_frequency = (td_matrix > 0).sum(axis=0) # Document frequency counts in how many documents each token appears

    if min_doc_freq is not None:
        td_matrix = td_matrix.loc[:, document_frequency >= min_doc_freq] # We only keep tokens if they appears a minimum of times. This removes very rare tokens that may not be informative.

    if max_doc_freq is not None:
        document_frequency = (td_matrix > 0).sum(axis=0)
        td_matrix = td_matrix.loc[:, document_frequency <= max_doc_freq] # We only keep tokens if they appears less than a maximum of times. This removes very common tokens that may be uninformative.

    vocabulary = td_matrix.columns.tolist() # Update the vocabulary to include only the tokens kept after filtering.

    return td_matrix, vocabulary

EXECUTION

In [ ]:
print("\n=== BUILDING TERM-DOCUMENT MATRICES ===")

if "MBG" in all_tokenized_dicts:
    print(f"{BLUE}[MBG] Building Matrix...{RESET}")
    MBG_td_matrix, MBG_vocab = build_term_document_matrix(all_tokenized_dicts["MBG"], min_doc_freq=4, max_doc_freq=400)
    print("MBG Matrix shape:", MBG_td_matrix.shape)
    print(MBG_vocab[:20])
    print(MBG_td_matrix)

if "wiki" in all_tokenized_dicts:
    print(f"\n{BLUE}[Wiki] Building Matrix...{RESET}")
    wiki_td_matrix, wiki_vocab = build_term_document_matrix(all_tokenized_dicts["wiki"], min_doc_freq=2)
    print("Wiki Matrix shape:", wiki_td_matrix.shape)
    print(wiki_vocab[:20])
    print(wiki_td_matrix)

In [ ]:
print(f"\n=== MERGING DATASETS ===")

all_dfs = []

for source in sources:
    f_token = get_filenames(source["name"])["tokenized"]
    
    try:
        df = pd.read_csv(f_token)
        df['source_origin'] = source["name"] # On garde une trace de l'origine !
        all_dfs.append(df)
    except:
        print(f"Warning: {f_token} missing")

df_global = pd.concat(all_dfs, ignore_index=True)
df_global.to_csv("GLOBAL_dataset_tokenized.csv", index=False)
print(f"{GREEN}SUCCESS ! Final data in GLOBAL_dataset_tokenized.csv{RESET}\n")

🌟 2) Règle générale (recommandée en NLP)
Pour un corpus de taille N documents :

✔️ min_doc_freq ≈ 0.5% à 1% de N
✔️ max_doc_freq ≈ 40% à 60% de N

## TF-IDF

IMPLEMENTATION OF DEFINITIONS

In [1]:
def tf_idf(td_matrix) : # Calculate the TF-IDF matrix from the Term-Document matrix

    row_sums = td_matrix.sum(axis=1) # Compute the total number of tokens in each document (= sum on the td_matrix's rows)
    tf = td_matrix.div(row_sums, axis=0) # Compute Term Frequency (TF). Each term count is divided by the total number of tokens in the corresponding document (normalization). This accounts for differences in document length.
    df = (td_matrix > 0).sum(axis=0) # Compute Document Frequency (DF). For each term (column of the td_matrix), count the number of documents in which the term appears at least once.
    N = td_matrix.shape[0] # Compute the total number of documents. This corresponds to the number of rows in the td_matrix.
    idf = np.log((N) / (df)) # Compute the IDF (Specificity). Terms that appear in many documents receive a lower IDF, while rare terms receive a higher IDF, making them more informative.
    tf_idf = tf.mul(idf, axis=1) # Multiply each item in the tf column by the IDF corresponding so that each term's TF is weighted by its IDF score.

    return(tf_idf)

In [2]:
# To help us visualize, we transform the urls into readable labels
def url_to_label(url):

    last_part = url.rstrip('/').split('/')[-1] # We only keep the last part of the url
    label = ' '.join([word.capitalize() for word in last_part.replace('-', ' ').replace('_', ' ').split()]) # We delete unwanted element ("_") and replace them by a space. The following letter is set as a capital letter.

    return label

EXECUTION

In [ ]:
print("\n=== VISUALIZATION MBG ===")

""" FOR MINDBODYGREEN """

# 1. Calcul TF-IDF
MBG_tf_idf = tf_idf(MBG_td_matrix)
print(MBG_tf_idf)

# 2. Mandatory cleaning (Infinity -> NaN -> 0)
MBG_tf_idf_clean = MBG_tf_idf.replace([np.inf, -np.inf], np.nan).fillna(0)    

# 3. PCA (Réduction à 2 dimensions)
pca = PCA(n_components=2)
MBG_tf_idf_2d = pca.fit_transform(MBG_tf_idf_clean)

# 4. DataFrame pour faciliter le plot
MBG_tf_idf_2d_df = pd.DataFrame(MBG_tf_idf_2d, columns=['x', 'y'], index=MBG_tf_idf_clean.index)
MBG_tf_idf_2d_df['label'] = MBG_tf_idf_2d_df.index.map(url_to_label)

# 5. Plot
plt.figure(figsize=(8,6))
plt.scatter(MBG_tf_idf_2d_df['x'], MBG_tf_idf_2d_df['y'], color='blue')

# Ajout des labels
for _, row in MBG_tf_idf_2d_df.iterrows():
    plt.text(row['x']+0.003, row['y']+0.003, row['label'], fontsize=7)

plt.title("TF-IDF Visualisation")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

# Sauvegarde au lieu d'afficher (plus sûr pour un script)
output_img = "MBG_pca_visualization.png"
plt.savefig(output_img)
print(f"{GREEN}Graph saved to {output_img}{RESET}")

y'avait pas la suite pour wiki j'ai pas osé ajouter

In [ ]:
print("\n=== VISUALIZATION WIKI ===")

""" FOR WIKIPEDIA """

# 1. Calcul TF-IDF
wiki_tf_idf = tf_idf(wiki_td_matrix)
print(wiki_tf_idf)

## Similarity Matrix

IMPLEMENTATION OF DEFINITIONS

In [ ]:
def similarity_matrix(tf_idf):
    #tf_idf_clean = tf_idf.dropna(axis=0, how='any') # Remove rows with NaN values 
    #tf_idf_clean = tf_idf_clean[tf_idf_clean.sum(axis=1) != 0] # Remove rows that are entirely zero (empty vectors)

    tf_idf_clean = tf_idf.replace([np.inf, -np.inf], np.nan).fillna(0) # Replace "NaN" with a 0

    # Compute the cosine similarity matrix
    similarity_matrix_tfidf = cosine_similarity(tf_idf_clean) # Each value ranges from 0 (no similarity) to 1 (identical documents).

    # Convert to a DataFrame for better readability (optional)
    similarity_df_tfidf = pd.DataFrame(similarity_matrix_tfidf, index=tf_idf_clean.index, columns=tf_idf_clean.index) # Rows and columns are labeled with document identifiers (index)

    # Display the similarity DataFrame
    display(similarity_df_tfidf)
    return(similarity_df_tfidf)

In [ ]:
def plot_similarity_matrix(similarity_df):
    # Plot the similarity matrix
    plt.figure(figsize=(14, 11))
    plt.imshow(similarity_df, interpolation='nearest', cmap='viridis')
    plt.colorbar(label='Cosine Similarity')
    plt.title('Document Similarity Matrix')

    # Labels = [url_to_label(url) for url in similarity_df.columns] # We labelized the document not with their full link but their label found with the function (url_to_label)
    
    ax = plt.gca()  # We set the axis to have them set at the right place
    # ax.set_xticks(range(len(labels)))
    # ax.set_yticks(range(len(labels)))
    # ax.set_xticklabels( labels, rotation=45, ha='right', rotation_mode='anchor', fontsize=7)
    # ax.set_yticklabels( labels, fontsize=7)

    ax.set_xticks([]) 
    ax.set_yticks([])
    
    ax.set_xlabel('Documents')
    ax.set_ylabel('Documents')

    # Optionally, annotate the cells with similarity values
    """for i in range(len(similarity_df)):
        for j in range(len(similarity_df)):
            ax.text(j, i, f"{similarity_df.iloc[i, j]:.3f}", ha='center', va='center', color='white')"""

    plt.tight_layout()

EXECUTION

In [ ]:
MBG_similarity_matrix = similarity_matrix(MBG_tf_idf)
print(MBG_similarity_matrix)

In [ ]:
plot_similarity_matrix(MBG_similarity_matrix)

In [ ]:
wiki_similarity_matrix = similarity_matrix(wiki_tf_idf)
print(wiki_similarity_matrix)
plot_similarity_matrix(wiki_similarity_matrix)

In [ ]:
# Sans les modifs de chat (= le code du prof) mais alors les axes déconne un peu
# Compute the cosine similarity matrix
similarity_matrix_tfidf = cosine_similarity(tf_idf) # Each value ranges from 0 (no similarity) to 1 (identical documents).

# Convert to a DataFrame for better readability (optional)
similarity_df_tfidf = pd.DataFrame(similarity_matrix_tfidf, index=tf_idf.index, columns=tf_idf.index) # Rows and columns are labeled with document identifiers (index)

# Display the similarity DataFrame
display(similarity_df_tfidf)

def plot_similarity_matrix(similarity_df):
    # Plot the similarity matrix
    plt.figure(figsize=(14, 11))
    plt.imshow(similarity_df, interpolation='nearest', cmap='viridis')
    plt.colorbar(label='Cosine Similarity')
    plt.title('Document Similarity Matrix')

    labels = [url_to_label(url) for url in similarity_df.columns] # We labelized the document not with their full link but their label found with the function (url_to_label)
    
    plt.xticks(ticks=range(len(similarity_df.columns)), labels = labels, rotation=45, fontsize=7)  
    plt.yticks(ticks=range(len(similarity_df.index)), labels = labels, fontsize=7)
    plt.xlabel('Documents')
    plt.ylabel('Documents')

    # Optionally, annotate the cells with similarity values
    for i in range(len(similarity_df)):
        for j in range(len(similarity_df)):
            plt.text(j, i, f"{similarity_df.iloc[i, j]:.3f}", ha='center', va='center', color='white')

    plt.tight_layout()

plot_similarity_matrix(similarity_df_tfidf)


## Most frequent words

In [ ]:
all_tokens = [tok for tokens in tokenized_MBG_csv["tokenized_text"] for tok in tokens] # Create a list with all the tokens
MBG_word_freq = Counter(all_tokens) # Count the number of times a token appears in the list
MBG_word_freq.most_common(20) # Print only the 20 most frequent

In [ ]:
all_tokens = [tok for tokens in tokenized_wiki_csv["tokenized_text"] for tok in tokens] # Create a list with all the tokens
wiki_word_freq = Counter(all_tokens) # Count the number of times a token appears in the list
wiki_word_freq.most_common(20) # Print only the 20 most frequent

## Words cloud

In [ ]:
wc = WordCloud( width=900, height=450, background_color="white").generate_from_frequencies(MBG_word_freq)

plt.figure(figsize=(10,5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wc = WordCloud( width=900, height=450, background_color="white").generate_from_frequencies(wiki_word_freq)

plt.figure(figsize=(10,5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

## Analyse thématique MBG

In [ ]:
MBG_tf_idf_clean = MBG_tf_idf.replace([np.inf, -np.inf], np.nan).fillna(0)

"""Pourquoi c’est bien ?
K‑means et PCA refusent les NaN → tu évites les crashs

Tu garantis un espace vectoriel propre

Tu évites les erreurs silencieuses"""

In [ ]:
def plot_elbow(tf_idf, max_k=10):
    inertias = []
    n_docs = tf_idf.shape[0]
    max_k = min(max_k, n_docs - 1)
    K_values = range(2, max_k)

    for k in K_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(tf_idf)
        inertias.append(kmeans.inertia_)

    plt.figure(figsize=(8,5))
    plt.plot(K_values, inertias, marker='o')
    plt.xlabel("Nombre de clusters (k)")
    plt.ylabel("Inertia (distortion)")
    plt.title("Méthode du coude (Elbow Method)")
    plt.grid(True)
    plt.show()

    # approx du "knee" : le point où la diminution ralentit le plus
    # on prend le point avec la plus grande "différence d'inertie" successive
    deltas = np.diff(inertias)
    best_k_idx = np.argmin(deltas) + 1 # +1 car diff décale de 1
    best_k = K_values[best_k_idx]
    best_inertia = inertias[best_k_idx]

    return best_k, best_inertia, K_values, inertias


In [ ]:
best_k, best_inertia, K_values, inertias = plot_elbow(MBG_tf_idf_clean, max_k=10)
print("K idéal :", best_k)
print("Inertia :", best_inertia)

In [ ]:
def plot_silhouette(tf_idf, max_k=10):
    scores = []
    n_docs = tf_idf.shape[0]
    max_k = min(max_k, n_docs - 1)
    K_values = range(2, max_k)

    for k in K_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(tf_idf)
        score = silhouette_score(tf_idf, labels)
        scores.append(score)

    plt.figure(figsize=(8,5))
    plt.plot(K_values, scores, marker='o')
    plt.xlabel("k")
    plt.ylabel("Silhouette score")
    plt.title("Méthode du Silhouette Score")
    plt.grid(True)
    plt.show()

    best_score = max(scores) 
    best_k = K_values[scores.index(best_score)] 
    return best_k, best_score, list(K_values), scores

In [ ]:
best_k, best_score, ks, scores = plot_silhouette(MBG_tf_idf_clean) 
print("Best k :", best_k) 
print("Associated silhouette score :", best_score)

Attention : on prend 5 pour k et pas 4 parce que les clusters sont mesurés sur base de tokens (= pas de stopwords ou custom_stopwords) sauf que Bert est sur normalized_text et même si on prend dans vectorizer les stopwords, Bert ne les exclut pas donc on prend k+1 car un cluster sera pour les noms.

In [221]:
MBG_k = 5
MBG_kmeans = KMeans(n_clusters=MBG_k, random_state=42)
MBG_tf_idf_clean['cluster'] = MBG_kmeans.fit_predict(MBG_tf_idf_clean)

In [ ]:
pca = PCA(n_components=2)
coords = pca.fit_transform(MBG_tf_idf_clean.drop(columns=['cluster']))

MBG_pca_df = pd.DataFrame(coords, columns=['x','y'], index=MBG_tf_idf_clean.index)

plt.figure(figsize=(8,6)) 
plt.scatter(MBG_pca_df['x'], MBG_pca_df['y'], c=MBG_tf_idf_clean['cluster'], cmap='tab10') 
plt.title("Clusters K-means") 
plt.show()

#### Bert

In [ ]:
def bertopic(normalized_corpus, vectorizer_model, cluster_model, min_words=100, embedding_model_name="all-MiniLM-L6-v2", nr_words=100):
    """
    Pipeline BERTopic avec KMeans (sans HDBSCAN), en forçant BERTopic
    à retourner plus que 10 mots par topic grâce à KeyBERTInspired.
    """

    # Load corpus
    df = pd.read_csv(normalized_corpus)
    documents = df["normalized_text"].astype(str).tolist() # Make sure that every document is a string
    documents = [d for d in documents if len(d.split()) > min_words] # remove documents with less than a x words (they are too short and only makes noise)

    # Embedding model
    embedding_model = SentenceTransformer(embedding_model_name)

    # Representation model forcing >10 words
    representation_model = KeyBERTInspired(top_n_words=nr_words)

    # BERTopic model
    topic_model = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model, hdbscan_model=cluster_model, representation_model=representation_model, verbose=True)

    # Fit and transform
    topics, probs = topic_model.fit_transform(documents)

    return topic_model, topics, probs


In [ ]:
full_stop_words = list(set(stop_words).union(custom_stopwords))

vectorizer_model = CountVectorizer( stop_words=full_stop_words, min_df=4, max_features=5000)

MBG_bert, MBG_topics, MBG_topics_probs = bertopic( "data/normalized_MBG_corpus.csv", vectorizer_model=vectorizer_model, cluster_model=MBG_kmeans, min_words=100, embedding_model_name="all-MiniLM-L6-v2", nr_words=100)


In [ ]:
def summarize_topics(topic_model, top_words=3, max_words=100, min_score=0.1):
    """
    Résume les topics d'un modèle BERTopic :
    - Crée un nom simple pour chaque topic à partir des mots les plus importants
    - Note le nombre de documents par thème
    - Identifie le thème principal (topic avec le plus de documents)
    
    Parameters
    ----------
    topic_model : BERTopic
        Modèle BERTopic déjà entraîné.
    top_words : int
        Nombre de mots à utiliser pour créer le nom du thème.
    
    Returns
    -------
    themes : dict
        Dictionnaire {nom_du_thème : {"words": [...], "count": N_docs}}.
    main_theme : dict
        Infos du thème principal : {'topic_id', 'name', 'words', 'count'}.
    topic_df : pd.DataFrame
        DataFrame avec Topic ID, Count, Name.
    """
    
    # 1️⃣ Infos des topics
    topic_info = topic_model.get_topic_info() # Topic, Count, Name
    topic_info = topic_info[topic_info.Topic != -1] # ignorer les outliers

    main_topic_id = (
        topic_info.sort_values(by="Count", ascending=False)
        .Topic
        .iloc[0]
    )

    # 2️⃣ Créer le dictionnaire themes avec le nombre de documents
    themes = {}
    for topic_id in topic_info.Topic:
        words_scores = topic_model.get_topic(topic_id)
        # 🔥 filtrage par score + limite
        words = [
            word
            for word, score in words_scores
            if score >= min_score
        ][:max_words]
        # words = [w for w, _ in topic_model.get_topic(topic_id)]
        theme_name = ", ".join(words[:top_words])
        count_docs = topic_info.loc[topic_info.Topic == topic_id, "Count"].values[0]
        is_main_topic = (topic_id == main_topic_id)
        themes[theme_name] = {"keywords": words} 
        
        print(f"Theme: {theme_name}")                            
        print("Numbers of documents", count_docs)
        print(words)
        
        if is_main_topic:
            main_topic_words = words
            print("-> This is the main topic")

        print("-" * 40)

    
    # 4️⃣ Retourner un DataFrame complet
    topic_df = topic_info.copy()
    
    return themes, main_topic_words, topic_df

In [ ]:
themes, main_topic_words, topic_df = summarize_topics(MBG_bert, top_words=3)

In [324]:
def filter_topic_words(main_topic_words):
    filtered_words = []
    for word in main_topic_words :
        if len(word) <= 2 :
            continue
        if word in full_stop_words :
            continue
        filtered_words.append(word)
    return filtered_words

In [ ]:
main_topic_words_filtered = filter_topic_words(main_topic_words)
print(len(main_topic_words))
print(len(main_topic_words_filtered))
print(main_topic_words_filtered)

In [ ]:
# Créer un DataFrame
df = pd.DataFrame({"main_topic_words_filtered": [main_topic_words_filtered]}) # note les [ ] pour que ce soit UNE cellule

# Sauvegarder
df.to_csv("data/main_topic_words_filtered.csv", index=False)

#### LDA

In [ ]:
n_topics = MBG_k # Define the number of topics you want the LDA model to extract from the documents.
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42) # Create an LDA (Latent Dirichlet Allocation) model instance. random_state = seed for reproducibility (ensures the same results each time)

lda_model.fit(MBG_td_matrix) # LDA learns the topic distributions for each document and the word distributions for each topic. We use term-document matrix as the vector for the LDA matrix (!! not tf-idf).

In [ ]:
# A SUPPRIMER (??)

MBG_words = MBG_td_matrix.columns.tolist()
MBG_themes = {}

for topic_idx, topic in enumerate(lda_model.components_):
    distinctiveness = topic / lda_model.components_.sum(axis=0)
    top_idx = distinctiveness.argsort()[::-1][:3]
    topic_name = ", ".join(MBG_words[i] for i in top_idx)

    MBG_themes[topic_name] = [MBG_words[i] for i in topic.argsort()[::-1][:10]]

MBG_themes

In [ ]:
words = MBG_td_matrix.columns.tolist() # Get the list of words (terms) corresponding to the columns of the td_matrix.
themes = {} # Initialize an empty dictionary to store topics. Key = topic name (most frequent word), Value = set of top words for that topic.

for topic_idx, topic in enumerate(lda_model.components_): # For each topic in the LDA model. lda_model.components_ is a matrix of shape (n_topics, n_words). Each row corresponds to a topic and contains the importance of each word for that topic.
    
    top_words_idx = topic.argsort()[::-1][:10] # We keep only the 10 main words that have the most relevance (topic.argsort() sorts word indices in ascending order by their importance)
    top_words = [words[i] for i in top_words_idx] # Convert the indices into actual word strings using the 'words' list.

    topic_name = top_words[0] # We give a "name" to the topic (which is the most frequant word of the list)

    """topic_word_scores = lda_model.components_[topic_idx]
    distinctiveness = topic_word_scores / lda_model.components_.sum(axis=0)
    top_idx = distinctiveness.argsort()[::-1][:1]
    topic_name = ", ".join(words[i] for i in top_idx)"""

    themes[topic_name] = set(top_words) # Store the topic in the dictionary.

print(themes) 

for name, words_set in themes.items():
    print(f"{name}: {words_set}") # For a clearer view of the topics and their corresponding words

In [ ]:
# Creates a Word cloud for every topic

words = MBG_td_matrix.columns.tolist()

for topic_idx, topic in enumerate(lda_model.components_):
    # Get the top words and their scores
    top_words_idx = topic.argsort()[::-1][:20] # top 20 words
    top_words = [words[i] for i in top_words_idx]
    word_freq = {words[i]: topic[i] for i in top_words_idx} # word -> importance

    topic_name = top_words[0]
    print(topic_name)

    # Create the WordCloud
    wc = WordCloud( width=900, height=450, background_color="white").generate_from_frequencies(word_freq)

    # Display the WordCloud
    plt.figure(figsize=(6, 3))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(topic_name, fontsize=16, color='black') # Topic name at the top
    plt.show()


Trouver les thèmes les pluc récurrents et les noms qu'on devrait leur donner

In [ ]:
doc_topic_matrix = lda_model.transform(MBG_td_matrix)
theme_strength = doc_topic_matrix.sum(axis=0)
theme_order = theme_strength.argsort()[::-1]
print(theme_order)
print(theme_strength)

In [ ]:
""" Nommer le thème avec le mot le plus “distinctif”
Au lieu de prendre le mot le plus fréquent dans le thème, tu prends le mot qui :

est très important dans ce thème
mais peu important dans les autres thèmes
C’est ce qu’on appelle un mot discriminant. """

topic_word_scores = lda_model.components_[topic_idx]
distinctiveness = topic_word_scores / lda_model.components_.sum(axis=0)
top_idx = distinctiveness.argsort()[::-1][:4]
topic_name = ", ".join(words[i] for i in top_idx)
print(topic_name)

In [ ]:
# Nommer le thème avec les documents qui lui appartiennent
# Tu regardes les documents où ce thème est dominant, puis tu en extrais les mots les plus fréquents

doc_topic = lda_model.transform(MBG_td_matrix)
dominant_docs = MBG_td_matrix.iloc[doc_topic[:, topic_idx].argsort()[::-1][:5]]
top_words = dominant_docs.sum().sort_values(ascending=False).head(4).index
topic_name = ", ".join(top_words)
print(topic_name)

In [ ]:
# ANALYSE CROISÉE K‑MEANS ↔ LDA

doc_topic = lda_model.transform(MBG_td_matrix)
"""LDA renvoie une matrice document → thèmes.
Chaque ligne = un document
Chaque colonne = un thème
Chaque valeur = probabilité que le document appartienne à ce thème"""

lda_dominant = doc_topic.argmax(axis=1) 
"""Pour chaque document, on prend le thème dominant (celui avec la probabilité la plus haute)."""

pd.crosstab(MBG_tf_idf_clean['cluster'], lda_dominant)
"""On compare :
les clusters K-means (géométriques)
les thèmes LDA (sémantiques)
C’est une matrice de correspondance."""


"""
col_0	0	1	2	3
cluster				
0	0	1	0	0
1	1	1	1	0
2	4	1	0	1

✔️ K-means et LDA ne racontent pas la même histoire
K-means regroupe selon la géométrie TF-IDF

LDA regroupe selon la distribution des mots

✔️ Ton tableau montre :
un cluster très cohérent (cluster 0)

un cluster très incohérent (cluster 1)

un cluster partiellement cohérent (cluster 2)

👉 Ton clustering n’est pas très aligné avec les thèmes LDA.

Ce n’est pas grave, mais ça montre que K-means n’est pas le meilleur outil pour détecter des thèmes dans un petit corpus."""

In [ ]:
topic_names = list(themes.keys())

doc_topic = lda_model.transform(MBG_td_matrix)
lda_dominant = doc_topic.argmax(axis=1) 
cross = pd.crosstab(MBG_tf_idf_clean['cluster'], lda_dominant)

cross.columns = [topic_names[i] for i in cross.columns]
cross.index.name = "Clusters K-means" 
cross.columns.name = "Thèmes LDA"

print(cross)

In [ ]:
# Comparaison automatique K‑means ↔ LDA

def compare_kmeans_lda(tfidf_df, lda_model, td_matrix):
    """
    Compare les clusters K-means et les thèmes LDA.
    Retourne :
    - une matrice croisée
    - une interprétation automatique
    """

    # 1. Thème dominant par document (LDA)
    doc_topic = lda_model.transform(td_matrix)
    lda_dominant = doc_topic.argmax(axis=1)

    # 2. Cluster K-means déjà dans tfidf_df['cluster']
    clusters = tfidf_df['cluster']

    # 3. Matrice croisée
    cross = pd.crosstab(clusters, lda_dominant)

    print("=== Matrice croisée K-means ↔ LDA ===")
    print(cross)
    print()

    # 4. Interprétation automatique
    print("=== Interprétation automatique ===")
    for cluster_id in cross.index:
        row = cross.loc[cluster_id]
        dominant_topic = row.idxmax()
        count = row.max()

        print(f"- Cluster {cluster_id} → Thème LDA dominant : {dominant_topic} ({count} documents)")

    return cross


In [ ]:
compare_kmeans_lda(MBG_tf_idf_clean, lda_model, MBG_td_matrix)

In [200]:
def get_theme_from_url(url, td_matrix, lda_model, topic_names):
    """
    Retourne le thème dominant associé à une URL déjà présente dans la td_matrix.
    
    url : str
        L'URL du document (clé de la td_matrix)
    td_matrix : DataFrame
        La matrice terme-document utilisée pour entraîner LDA
    lda_model : LatentDirichletAllocation
        Le modèle LDA déjà entraîné
    topic_names : list
        Liste ordonnée des noms de thèmes (ex: ["cortisol", "vo2", ...])
    """

    # Vérifier que l'URL existe dans la matrice
    if url not in td_matrix.index:
        raise ValueError(f"L'URL '{url}' n'existe pas dans la td_matrix.")

    # Extraire la ligne correspondant au document
    doc_vector = td_matrix.loc[url].values.reshape(1, -1)

    # Appliquer LDA sur ce document
    topic_distribution = lda_model.transform(doc_vector)[0]

    # Trouver le thème dominant
    dominant_topic_id = topic_distribution.argmax()

    # Retourner le nom du thème
    return topic_names[dominant_topic_id], topic_distribution

In [ ]:
get_theme_from_url("https://www.mindbodygreen.com/articles/journey-to-long-covid-diagnosis-and-how-starting-to-heal", MBG_td_matrix, lda_model, topic_names)

In [ ]:
def nodes_theme(input_csv, output_csv):
    df = pd.read_csv(input_csv) # Take the csv file with the raw html as the input

    if 'id' not in df.columns: # Verify that the html colon exists
        raise ValueError(f"The id column is missing in: {input_csv}")

    df['id'] = df['theme'].apply(get_theme_from_url) # Cleans the html column
    df = df[['id', 'label', 'theme']] # Keep the url and text colon (not the raw html)

    df.to_csv(output_csv, index=False, encoding='utf-8') # Creats a new csv file as the output of the function

    return df

## Analyse temporelle

In [ ]:
def count_themes(tokens, themes): # Count the number of times a theme (or associated word) appears in each document (/ doc's tokens)
    counts = Counter()
    token_set = set(tokens)
    for theme, keywords in themes.items():
        counts[theme] = len(token_set & keywords)
    return counts

In [ ]:
timeline_MBG = tokenized_MBG_csv.copy() # To be safer, we creat a copy of the dataframe
timeline_MBG ['date'] = pd.to_datetime(timeline_MBG['date'], errors='coerce') # Transforms the dates into correct dates frame and if the data is not convertible it becomes "Not a Time"
timeline_MBG = timeline_MBG.dropna(subset=['date']) # Delete all rows whith unvalid dates

In [ ]:
rows = []

for i, row in timeline_MBG.iterrows(): # For every line in the dataframe (timeline_MBG)
    tokens = row['tokenized_text'] # Collect the tokens in the correct column

    theme_counts = count_themes(tokens, themes) # Count the number of times the theme appears in the article's tokens

    for theme, count in theme_counts.items(): # For every theme and number of times it appears
        rows.append({ "date": row['date'],  "theme": theme, "count": count }) # Construction of a dictionnary for each date and for each theme and the number of times it appears

theme_timeline_MBG = pd.DataFrame(rows) # Transform the list of dictionaaries in a dataframe

In [ ]:
daily_evolution_timeline_MBG = ( theme_timeline_MBG
    .groupby(['date', 'theme'])['count'] # Group the datas (theme_timeline_MBG) according to their date and theme
    .sum() # If multiples rows of theme_timeline_MBG has the same group (= same date and theme) it is sum to form only one row
    .reset_index() # Important to not have the columns changed into date, theme (which is done automatically by groupby)
)

In [ ]:
# Pivot par jour
pivot = ( daily_evolution_timeline_MBG .pivot(index='date', columns='theme', values='count') .fillna(0)) # We pivot the table

# Plot
pivot.plot(figsize=(12,6))
plt.title("Daily evolution of themes in lifestyle blogs")
plt.xlabel("Date")
plt.ylabel("Occurrences")
plt.legend(title="Themes")
plt.tight_layout()
plt.show()


In [ ]:
# Lissage par moyenne mobile sur 7 jours  -> A supprimer
daily_smoothed = pivot.rolling(window=7).mean()

daily_smoothed.plot(figsize=(14,6))
plt.title("Évolution quotidienne lissée (moyenne mobile 7 jours)")
plt.xlabel("Date")
plt.ylabel("Occurrences moyennes")
plt.tight_layout()
plt.show()


In [ ]:
# Load corpus
df = pd.read_csv("data/normalized_MBG_corpus.csv")
documents = df["normalized_text"].astype(str).tolist()
documents = [d for d in documents if len(d.split()) > 100] # Remove documents with less than a 100 words (they are too short and only makes noise)

# Embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # It transforms every document in a vector which have meaning 

# Vectorizer
vectorizer_model = CountVectorizer(
    stop_words="english",
    min_df=3,
    ngram_range=(1, 2)
)

# KMeans clustering (NO hdbscan)  -> clustering of documents per topic (we can ask for x cluster)
cluster_model = KMeans(
    n_clusters=6,
    random_state=42,
    n_init="auto"
)

# BERTopic without hdbscan
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=cluster_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(documents)

print(topic_model.get_topic_info())

In [ ]:
pca = PCA(n_components=2) # We keep 2 dimensions to construct the graph
bert_2d = pca.fit_transform(topic_model) # Compute the vectors in 2 D

bert_2d_df = pd.DataFrame(tf_idf_2d, columns=['x', 'y'], index=topic_model.index) # Compute a dataframe pandas
bert_2d_df['label'] = bert_2d_df.index.map(url_to_label)

# Plot
plt.figure(figsize=(8,6))
plt.scatter(bert_2d_df['x'], bert_2d_df['y'], color='blue')

# To delete if we don't want the labels
for _, row in bert_2d_df.iterrows():
    plt.text(row['x']+0.003, row['y']+0.003, row['label'], fontsize=7)

plt.title("TF-IDF Visualisation")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()


In [ ]:
silhouette_scores = []
k_range = range(2, 16)

for k in k_range:
    kmeans = KMeans(
        n_clusters=k,
        random_state=42,
        n_init=10
    )
    labels = kmeans.fit_predict(topic_model)
    
    score = silhouette_score(topic_model, labels)
    silhouette_scores.append(score)

    print(f"k = {k} | Silhouette Score = {score:.4f}")

plt.figure(figsize=(8, 5))
plt.plot(k_range, silhouette_scores, marker='o')
plt.xlabel("Nombre de clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score pour différents nombres de clusters")
plt.grid(True)
plt.show()


# Link analysis sur text mining

In [205]:
wiki_nodes_path = r"C:\Users\Violaine\OneDrive\Ecole\Master 1\Web mining\Projet Web Mining\Projet_Web_Mining\data\wikipedia_nodes.csv"
wiki_edges_path = r"C:\Users\Violaine\OneDrive\Ecole\Master 1\Web mining\Projet Web Mining\Projet_Web_Mining\data\wikipedia_edges.csv"

wiki_nodes = pd.read_csv(wiki_nodes_path)
wiki_edges = pd.read_csv(wiki_edges_path)

In [ ]:
# ---------------------------
# 1️⃣ Charger les données
# ---------------------------

G = nx.DiGraph()
# Ajouter les nœuds avec l'attribut thème
for idx, row in wiki_nodes.iterrows():
    G.add_node(row['id'], theme=row['label']) # A changer par theme !!

# Ajouter les liens
for idx, row in wiki_edges.iterrows():
    G.add_edge(row['source'], row['target'])

In [ ]:
# Compter liens intra-thème et inter-thème  
intra = 0
inter = 0

for u, v in G.edges():
    if G.nodes[u]['theme'] == G.nodes[v]['theme']:
        intra += 1
    else:
        inter += 1

total_links = G.number_of_edges()
print(f"Liens intra-thème : {intra} ({intra/total_links:.2%})") # -> Homophilie thématique
print(f"Liens inter-thème : {inter} ({inter/total_links:.2%})")

In [ ]:
pagerank = nx.pagerank(G)
betweenness = nx.betweenness_centrality(G)
indegree = dict(G.in_degree())

## Analyse thématique Wikipedia